# Baseball Lab 9


Welcome to the ninth baseball lab. This lab is due by  **Monday April 26th at 11:30pm**. 

As always, you are welcome to work with others on the lab, but if you do work with others after class, please note on the reflection quiz who you worked with. If you have questions about the homework please use [Ed discussions](https://edstem.org/us/courses/4202/discussion/) so that everyone can benefit from your question. Additionally, please answer questions others raise on Ed discussions. Finally, please feel free to attend Neel's and my office hours for additional help. 




#### Today's Baseball Lab

In today's exercises, you'll get practice doing:

1. Simple linear regression analyses in Python
2. Multiple linear regression analyses in Python

Reading that will be useful for solving the problems: class textbook [chapter 15](https://www.inferentialthinking.com/chapters/15/Prediction.html) 


In [2]:
# importing packages that will be used in this lab

from datascience import *
import pandas as pd
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt


np.random.seed(173)

%matplotlib inline


# Part 1: Simple linear regression: predicting runs based on batting average

In the first part of this lab, we will build a linear regression model that can predict how many runs a team will score based on the team's batting average. The code below loads data on team-level statistics. There is also a function below called `add_derived_stats()` which takes a table with the teams' data and adds the statistics BA, SLG, OBP and OPS. 


In [6]:
# load the teams data...
teams = Table.read_table('https://raw.githubusercontent.com/emeyers/SDS173/master/data/Teams.csv')  


# a function that adds that statistics: BA, SLG, OBP, and OPS
def add_derived_stats(teams_data):
    
    # batting average: ba
    ba = teams_data['H']/teams_data['AB']

    # create slugging percentage
    x1b = teams_data['H'] - (teams_data['2B'] + teams_data['3B'] + teams_data['HR'])
    slg = (x1b + (2 * teams_data['2B'] + 3 * teams_data['3B'] + 4 * teams_data['HR']))/teams_data['AB']

    # create on-base percentage
    obp = (teams_data['H'] + teams_data['BB'] + teams_data['HBP'])/(teams_data['AB'] + teams_data['BB'] + teams_data['HBP'] + teams_data['SF'])

    # create on-base plus slugging percentage
    ops = obp + slg

    # create the table that has all the desired statistics
    teams_with_addition_stats = teams_data.with_column('1B', x1b).with_column('BA', ba).with_column('SLG', slg).with_column('OBP', obp).with_column('OPS', ops)

    return teams_with_addition_stats




# # If you are using Pandas you can use this code instead of the code above

# teams = pd.read_csv('https://raw.githubusercontent.com/emeyers/SDS173/master/data/Teams.csv')  


# # a function that adds that statistics: BA, SLG, OBP, and OPS
# def add_derived_stats(teams_data):
    
#     # batting average: ba
#     ba = teams_data['H']/teams_data['AB']

#     # create slugging percentage
#     x1b = teams_data['H'] - (teams_data['2B'] + teams_data['3B'] + teams_data['HR'])
#     slg = (x1b + (2 * teams_data['2B'] + 3 * teams_data['3B'] + 4 * teams_data['HR']))/teams_data['AB']

#     # create on-base percentage
#     obp = (teams_data['H'] + teams_data['BB'] + teams_data['HBP'])/(teams_data['AB'] + teams_data['BB'] + teams_data['HBP'] + teams_data['SF'])

#     # create on-base plus slugging percentage
#     ops = obp + slg

#     # create the table that has all the desired statistics
#     teams_with_addition_stats = teams_data.copy()
#     teams_with_addition_stats["1B"] = x1b
#     teams_with_addition_stats["BA"] = ba
#     teams_with_addition_stats["SLG"] = slg
#     teams_with_addition_stats["OBP"] = obp
#     teams_with_addition_stats["OPS"] = ops
    
#     return teams_with_addition_stats


## 1.1: Creating a plot with a regression line for 2013 data

**Exercise 1.1 (5 points)**: Let's start by extracting the data for the 2013 season and using the `add_derived_stats()` function to add additional statistics to this table. Then create a scatter plot that plots runs as a function of batting average and add the regression line to this plot using the `fit_line = True` argument to the scatter plot method (if you are using Pandas, you can skip adding the regression line to your plot for this problem). 


## 1.2: Using the Statsmodels formula interface to get the regression coefficients

Let's now use the Statsmodels package with the "formula" interface to build a linear regression model. We can use the ordinary least-squares method (ols) which has the form `lm = smf.ols('y ~ x', data = my_table)`, where the arguments to this method are:

- `y ~ x`: a string that specifies the column names of the y and x variables in the regression model
- my_table: a table (or pandas DataFrame) that contains the data

The output of this method is an object that implements the RegressionResults interface which we can use to get the regression coefficients (i.e., the coefficients found based on doing a least-squares fit). 

**Exercise 1.2 (5 points)**: Create a linear regression model that predicts runs based on batting average. Save the model with the name `lm`. Then use `lm.params` to get the intercept and slope parameters. Save these parameters to the name `the_params` and print these parameter values. Finally, in the answer section below, write out the equation for the regression line using LaTeX. Hint: some of this LaTeX code will be useful in this an subsequent exercises: $\hat{z}$, $\cdot$, $\beta$, $z_1$. 


**Answer**






In [1]:
# using the statsmodels formula notation 
import statsmodels.formula.api as smf 

# create a fitted model in one line


# print the coefficients




## 1.3: Making predictions using the linear regression model

Now that we have the regression coefficients, we are able to make predictions for new x values. In particular, for the model we have created above, we can predict the number of runs a team will score based on a team's batting average. 

To do this using the Statsmodels package we can use the `lm.predict(tb_x_vals)` method. This method takes a table as an argument, `tb_x_vals`, that has a column of x values. The name of this column must be the same as the column name that was used for the x values when the model was fit (i.e., 'BA' in our example). From these x values, the `predict()` method calculates corresponding $\hat{y}$ predictions using the linear regression coefficients that were found.

**Exercise 1.3 (7 points)**: Create a table called `ba_values` that has a single column called 'BA'. This column should have sequential values from .220 to .290 at increments of .001 (hint: using the `np.arange()` function will be useful here). Once you have created that table, get the corresponding $\hat{y}$ values using the `lm.predict` method and store these in the name `run_predictions`. 

Next recreate the scatter plot for the 2013 data you made in exercise 1.1 but do not use the `fit_line = True` argument. Instead, manually add the regression line by using `plt.plot(x, y)` for the BA values in the `ba_values` table and the corresponding predictions in the `run_predictions`.



## 1.4: Calculating the residuals

Now that we have a function that can give us the predicted values $\hat{y}$ from the regression line, we can use it to assess how far off these predictions are using *the data that was used to fit the model*. The difference between a real value $y_i$ and the predicted value $\hat{y_i}$ is called a residual, and we can use these residuals to assess how closely our model fits the data. 

**Exercise 1.4 (point 6):** Use the following steps to calculate the residuals: 

1. Use the lm.predict() method to get the predicted $\hat{y}$ from the 2013 data. Save the results to the name `predictions_df`.

2. `predictions_df` is a data type called a Pandas Series (you can verify this by running the command `type(predictions_df)`. Let's convert this to a numpy ndarray using the `predictions_df.to_numpy()` method and save the results to the name `predictions_np`. 

3. Calculate the residuals for each team and store them in the name `residuals`. Also print these values.

4. Finally, report the sum of all the residuals. What value do you get? 


**Answer**: What value do you get for the sum of the residuals? 





## 1.5: Calculating the root mean squared error

One use of the residuals is to use them to tell us how well the regression line fits the data. We can do this by computing the "mean squared residual" (MSE) using the formula: 

$MSE = \frac{1}{n}\sum_i^n {(y_i - \hat{y_i})^2}$

If we take the square root of the MSE, we get the "root mean squared residual" (RMSE) which tells us how far, on average, the predictions from the regression line are from the actual y values. 

**Exercise 1.5 (6 points):** Calculate the MSE and the RMSE below and save them to the name `mse` and `rmse`. Then print these values. On average, how many runs off are the predictions? 

**Answer** Report how far off, on average, the predictions are for the 2013 data in terms of runs.




## 1.6: Relating the RMSE to the correlation coefficient r

In last class and on homework 8, we showed that the correlation coefficient, r, can be used to assess how good a statistic is. We can also use the root mean squared error as a measure to determine how good a statistic is at predicting runs. It turns out that using these two methods are equivalent since they are related by the formula: 

$r^2 =  1 - \frac{MSE}{s_y}$

where we have: 

- $r^2$: is the squared correlation coefficient
- MSE: is the mean squared error
- $s_y$: is the variance of y computed by a method that does not correct for bias, i.e. $s_y = \frac{1}{n}\sum_i^n (y_i - \bar{y}^2)$. This variance formula that does not correct for bias is the formula used by the numpy's `np.var()` function. 

Note: almost all statisticians use the bias corrected formula for the variance which is: $s_y = \frac{1}{n-1}\sum_i^n (y_i - \bar{y}^2)$, but for some reason numpy has decided to use the biased formula. For reasonably large n, however, the difference is negligible. 


**Exercise 1.6 (6 points):** Calculate the value of $r^2$ and the value of $1 - \frac{MSE}{s_y}$ for relating/predicting runs from batting average. Show that they are the same (to at least 10 decimal places) by printing these values. 


# Part 2: Additional simple linear regression exercises

Let's explore simple linear regression a little further with a few additional exercises. 


## 2.1: getting a larger data set

 
For these exercises let's look at data from a slightly larger data set by using all team seasons from 1970 where the teams played 162 games (this is the same data set you used on lab 8). 

**Exercise 2.1 (5 points):** Get the data below and assign it to the name `teams_162`. Also use the `add_derived_stats()` function to add BA, SLG, OBP and OPS to this table. Print the number of rows in the table, which should be 1019, to make sure you have done this correctly. Also, create a scatter plot of runs as a function of the number of home runs teams hit with the regression line on it. 



## 2.2: Predicting the number of runs a team will score based on the number of HR 



**Exercise 2.2 (6 points):** Now using the `teams_162` table, create a model that predicts the number of runs a team will score based on the number of home runs hit. Save the coefficients from this model in the name `the_params` and print out the coefficient values from this model. Then, in the answer section below, write out the equation for predicting the number of runs based on the number of home runs. Also, answer the questions based on the fit model: 

1. How many additional runs does a team score for every additional home run?
2. How many runs would you expect a team to score that hit no home runs? 


**Answer:**


1. 
2. 



In [71]:

# create a fitted model in one line



# print the coefficients





Intercept    498.151812
HR             1.477128
dtype: float64

## 2.3: writing your own function to make predictions


In exercise 1.3 you used the `lm.predict()` method to make predictions. Now let's write our own function to make predictions. 


**Exercise 2.3 (8 points):** Please write a function called `make_predictions(the_coeffs, x_vals)` that takes the following arguments: 

1. `the_coeffs`: an ndarray like object (or Pandas series) of coefficients (slope and intercept). These should be the values returned by lm.params 

2. `x_vals`: an ndarray of values from which the predictions should be made


Once you have written this function, do the following steps:

1. Use the make_predictions() function to make predictions for the number of home runs expected from the teams_162 data and save this to the name `my_hr_predictions`. 

2. Use the Statsmodel predict() method to make predictions of runs based on home runs and store the results in the name `sm_hr_predictions` as a numpy ndarray. 

3. Finally, subtract `my_hr_predictions` from `sm_hr_predictions`, take the absolute value of these differences, and sum them together. The value should be close to 0, showing that your function is giving the same results as the Statsmodel predict() method.


In [3]:

def make_predictions(the_coeffs, x_vals):
    pass







## Exercise 2.4:  Calculating the RMSE for different statistics

In exercise 1.6 you showed that the root mean squared error (RMSE) and the correlation coefficient (r) are essentially equivalent to each other in the sense that there is a one-to-one monotonic transformation from one value to the other. However, one reason in some circumstances one might prefer to use the RMSE is that it is reported in the same units as the original y variables. Since we have been predicting runs, the unit of RMSE is runs. 

Let's now compare our different statistics of HR, BA, OBP, SLG and OPS based on their RMSE values. 


**Exercise 2.4 (10 points)**: Write a function `get_rmse(the_data, y_col, x_col)` that takes the following arguments:

1. `the_data`: a data table

2. `y_col`: a string specifying a column name in `the_data` table that will be used for the y values in the regression

3. `x_col`: a string specifying a column name in `the_data` table that will be used for the x values in the regression


This function should do the following steps:

1. It should run a linear regression for predicting y as a function of x. Hint you can append strings together using the + symbol, i.e., 'a' + '~' + 'b'. 

2. It should then make predictions for the data the model was fit on.

3. It should calculate the RMSE from the residuals and return this value.


Once you have written this function, run it on the teams_162 data to get the RMSE between runs and HR, BA, OBP, SLG and OPS. Print out the values, on average, how far off each of these statistics are in terms of their ability to predict runs through a linear model. To put these values in perspective, also print the range of runs a team scored in a season (i.e., the maximum and minimum number of runs a team scored), and also look at the range of values in the scatter plot you created in 2.1 for some perspective.


In [4]:

def get_rmse(the_data, y_col, x_col):
    
    pass
    


    

    
    





# Part 3: Multiple regression

In multiple linear regression we use multiple predictions $x_i$ and multiple coefficients $\hat{\beta_i}$ to predict a value $y_i$, i.e., we use data to find the $\hat{\beta_i}$ in the equation $\hat{y_i} = \hat{\beta_0} + \hat{\beta_1} \cdot x_1 + \hat{\beta_2} \cdot x_2 + ... + \hat{\beta_k} \cdot x_k$. Once we have "learned" the coefficients $\hat{\beta_i}$ from a dataset (usually by minimizing the RMSE) we can then use this equation to make new predictions $\hat{y_i}$ for different $x_i$.

In the following exercise, you will use multiple regression to try to learn a new statistic that can predict runs better than all the other statistics we have seen so far (i.e., better than OPS). 



## 3.1:  Dealing with the particularities Statsmodel package

In order to find a better statistic for predicting runs, we will create a model that is a linear combination of basic statistics that has the form: $\hat{r} =  \hat{\beta_0} + \hat{\beta_1} \cdot 1B + \hat{\beta_2} \cdot 2B + ... + \hat{\beta_k} \cdot HR$, where $\hat{r}$ is the predicted number of runs a team scored in a season and $1B$, $2B$, ... $HR$ are the number of singles, doubles, ..., home runs, a team had in a season. 

To do this using the Statsmodels package in Python, we can use the formula interface using the syntax `'y = x1 + x2 + x3'` etc. However, Statsmodels package cannot handle variable names that start with numbers. Thus, if we tried to do an equation of the form: `'R ~ 1B + 2B + 3B'` we would get an error message. To deal with this, we need to rename our variables that start with numbers to instead start with letters. 

**Exercise 3.1 (5 points):** Please use the `tb.relabel()` method to relabel `1B` as `X1B`, `2B` as `X2B`, and `3B` as `X3B` in the `teams_162` dataset. Save this to a new name called `teams_162_relabel`. Note: the relabel method actually modifies the original dataset, so be careful if you run this method more than once! i.e., you might have to recreate the `teams_162` dataset if you run `tb.relabel` more than once.  

Print the first two rows of the `teams_162_relabel` to show that the relabeling has worked.


## 3.2: Building a multiple linear regression model to predict runs

Now that we have the data in a format that can be used by the Statsmodels package, let's build a linear regression model!


**Exercise 3.2 (8 points):** Please build a regression model that can predict runs based on the following predictors: walks (BB), hit by pitches (HBP), singles (X1B), doubles (X2B), triples (X3B), and home runs (HR). 

Once you have "learned" the regression coefficients, please print them out and write down the regression equation that you get in terms of BB, HBP, X1B, etc. Also answer whether the regression weights found by the model make sense to you in terms of their magnitudes, for both the intercept and the weights on each of the predictors. 

Finally, calculate the RMSE for the model you learned using the `get_rmse()` function you wrote in exercise 2.4. How does it compare to the RMSE for OPS? Note: if you have written the `get_rmse()` well, you should be able to pass a string for the x predictor values in the form `'x1 + x2 + x3'` etc., and it should still work! 


**Answer:**








## 3.3: How low can you go? 

Finally, try to build a multiple linear regression model that makes even more accurate predictions for the number of runs scored!

**Exercise 3.3 (9 points):**  Please include more predictors in your regression model to try to build a model that has the RMSE is as low as possible. Additionally, you can add more derived statistics to your `teams_162_relabel` data set, such as home runs squared, by adding new columns to this table. 

Once you have found a model with a low RMSE, print out the coefficients for this model and write out the model using LaTeX below, and comment on whether the weights found by your model make sense. We will compare the models you come up with next class and whoever creates the model with the smallest RMSE gets bragging rights!


**Answer**

The model I came up with is: 








## Part 4: Final project plan (3 points)

Please write a paragraph or two about what question your final project will try to answer and what data you plan to use for your project. Also, make sure you can load the data in Python, and I recommend you also start on some of the analyses for your project this week. 




## Part 5: Questions for Ben Reiter (3 points)

As you know, Ben Reiter will be joining us next class. To order to prepare a little for his visit, please write down three questions you would potentially be interested in asking him.

Question 1: 

Question 2: 

Question 3: 



# Part 6: Quote and reaction to Astroball chapter 9 (5 points)

Please find an interesting quote from chapter 9 Astroball and then write a ~one paragraph reaction to the quote below.

Quote: 

Reaction: 



# Part 7. Reflection (3 points)

How did this lab go? Please complete the [reflection homework 9](https://yale.instructure.com/courses/65116/quizzes/35184) which is under the quiz section on Canvas to let us know how it went and to reflect on what you learned. 
